In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## ARMA

En este tutorial abordaremos los modelos ARMA. Veremos diferentes maneras de identificarlos y modelarlos. Completarás un ejericio (Ejercicio 1) para ensayar; finalmente, desarrollaremos un ejercicio más completo donde generaremos un par de modelos a una serie ya estudiada con modelo MA(q), partiremos la serie para entrenamiento y prueba y mediante el pronóstico de la sección de prueba seleccionaremos el mejor modelo, el cuál servirá para generar un pronóstico a 10 días adelante.

##### Ejemplo ARMA.

**Contenido**
1. [Ejemplo 1: Datos y Modelamiento:](#1)
    1. [Importación de librerías requeridas](#2)
    1. [Importacion de datos](#3)
    1. [Análisis EACF](#4)
1. [Ejercicio 1 para completar](#5)
1. [Ejemplo de Clase, retomado del AM(9) restringido](#6)
    1. [Compararemos dos paqueterías para la estimación del modelo ARMA(p,q)](#7)
    1. [Modelo Sugerido por EACF](#9)
    1. [Utilizando otras librerías (auto.arima)](#10)
    1. [Grafica de los residuales](#11)
    1. [Comparación de desempeños con partición de datos](#12)
    1. [Particionando los datos](#13)
    1. [Training modelo con EACF](#14)
    1. [Training model, con auto arima:](#15)
    1. [Model summary](#16)
    1. [Evaluacion, con respecto a la muestra separada de validación](#18)
    1. [Bakctesting](#19)
    1. [Pronóstico hacia adelante mejor modelo](#20)



<a id="1"></a> <br>
## Ejemplo 1: Datos y Modelamiento

<a id="2"></a> <br>
### Importación de librerías requeridas

In [1]:
install.packages('TSA')
install.packages("Metrics")

In [2]:
library(TSA)  
library(quantmod)
library(forecast)
library(tseries)

library(Metrics)
options(warn = - 1) 

<a id="3"></a> <br>
###  Importacion de datos

<a id="4"></a> <br>
### Análisis EACF


<a id="5"></a> <br>
### Ejercicio 1 para completar

In [5]:
library(tseries)

### 1) Código para los rendimientos log de precio ajustado.

In [9]:
getSymbols("SPX",from="2000-01-02",to="2010-07-31", envir = .GlobalEnv) #period
sps= SPX %>% na.omit()
head(sps)
tail(sps)

##Ahora, analiza qué nivel de ARMA tendría.
#Previo a ello, debes preguntarte si puedes aplicar la función eacf a la sere directamente, 
#Qué condiciones crees que deba cumplir la serie?

eacf(?)

In [ ]:
#rendimientos
#partir la serie

Se toman los rendimientos ajustados para convertirlos rendimientos logarítmicos

In [12]:
getSymbols("SPX",from="2000-01-02",to="2010-07-31") #
head(SPX$SPX.Adjusted)

In [23]:
getSymbols("SPX",from="2000-01-02",to="2010-07-31", envir = .GlobalEnv) #period
sps= SPX %>% na.omit()
r1=periodReturn(sps$SPX.Adjusted, period="daily",  type="log")
head(r1)

In [24]:
eacf.r1=eacf(r1, 10, 10)

Se podría tomar un MA(1), otras posibilidades son un MA(4) y ARMA(3,1). Por tanto, a continuación se prueban estas opciones

In [15]:
mod1=Arima(r1,order=c(0,0,4)) #MA(4)

In [16]:
mod1

In [18]:
mod2=Arima(r1,order=c(3,0,1)) #ARMA(3,1)
mod2

In [20]:
mod3=Arima(r1,order=c(0,0,1)) #MA(1)
mod3

In [21]:
#La función de auto.arima de la paquetería de forecast()
mod_auto=auto.arima(r1, seasonal=FALSE)
summary(mod_auto)

se sugiere un MA(1) con el auto.arima

El modelo que derivamos a partir del método de eacf es más competitivo que el derivado por la función auto.arima(), ya que los valores de AIC son menores. Es decir el MA(4) es el de mejor ajuste en este caso.

Evaluación de parámetros del modelo MA(4)

In [22]:
tsdiag(mod1)

In [25]:
#Echemos un vistazo nuevamente a nuestra serie, donde son 2607 datos.
head(r1)
tail(r1)
l=length(r1)
l

### 2) Partición serie (h=10 datos para testing )

In [75]:
#reservemos un h=10 para el testing
l=length(r1)
l
h=10
entrena = r1[1:(l-h)] # se toma la serie menos h=10
valida = r1[(l-h+1):l] #se dejan los 10 datos finales

<a id="7"></a> <br>
#### A. Compararemos dos paqueterías para la estimación del modelo ARMA(p,q).


Generamos tabla simplificada


In [11]:
eacf.r1=eacf(r1,10,10) 
#Qué nivel sugieres es la serie?

<a id="9"></a> <br>
#### B. Modelo Sugerido por EACF


In [12]:
# Según la tabla eacf, existen varios modelos candidatos, podrían ser MA (1), ARMA(2,2), también el arma(4,3)
# o también, el modelo MA(3), que empleamos como modelo anteriormente.
#Propongamos el ARMA(2,2)
mod1=Arima(r1,order=c(2,0,2))

In [13]:
mod1

Como puntos relevantes, debemos mirar el valor del AIC.

<a id="10"></a> <br>
#### C. Utilizando otras librerías (auto.arima)

Empleemos otras librerías para tener otros modelos candidatos.

In [14]:
#La función de auto.arima de la paquetería de forecast()
mod=auto.arima(r1, seasonal=FALSE)
summary(mod)

Vemos que el modelo que derivamos a partir del método de eacf, es más competitivo que el derivado por la función auto.arima(), ya que los valores de AIC son menores.

<a id="11"></a> <br>
#### D. Gráfica de los residuales

Analicemos los residuales del modelo:


In [15]:
tsdiag(mod)

Obviemos el ajuste de residuales mediante la prueba de Xi cuadrada, ya que los valores p, son muy elevados

<a id="12"></a> <br>
### E. Comparación de desempeños con partición de datos

Ahora, ya teniendo un modelo ganador, apliquémoslo a una sección de entrenamiento y reservemos la sección de prueba.


<a id="14"></a> <br>

#### G. Training modelo con EACF

In [27]:
#Usualmente los modelos pueden tener cambios cuando cambiamos la serie, podemos aplicar nuevamnete
#la función eacf para derivar modelo. En otros paquetes, como el ModelTime() que veremos posteriormente
#este paso se llama calibración.
eacf(entrena)

Vemos que un modelo sería un MA(1), aunque también se debería probar un ARMA(1,1)

In [29]:
#Generamos modelo 1 a partir del eacf"
modelo1=Arima(entrena,order=c(0,0,1))

In [30]:
#Lo analizamos, revisamos residuales y generamos pronóstico:
modelo1
tsdiag(modelo1)


In [56]:
#Generamos modelo 2 a partir de la otra opción del eacf, en su caso un ARMA(1,3)"
modelo2=Arima(entrena,order=c(2,0,3))

In [65]:
#Le analizamos también sus residuales y generamos pronóstico:
modelo2
tsdiag(modelo2)


In [54]:
#Generamos modelo 3 a partir de la otra opción del eacf, en su caso un MA(4)"
modelo3=Arima(entrena,order=c(0,0,4))

In [55]:
#Le analizamos también sus residuales y generamos pronóstico:
modelo3
tsdiag(modelo3)

In [76]:
#Generamos modelo 4 a partir de la otra opción del eacf, en su caso un ARMA(1,1)"
modelo4=Arima(entrena,order=c(1,0,1))

In [77]:
#Le analizamos también sus residuales y generamos pronóstico:
modelo4
tsdiag(modelo4)

Probamos otra opción mediante un auto.airma

In [58]:
modelo1_auto=auto.arima(entrena, seasonal=FALSE)
modelo1_auto

La función auto.arima también nos sugiere un MA(1), por tanto, elegimos trabajar con el ARMA(2,3) para el pronóstico de los datos de entrenamiento

In [66]:
#Generamos pronóstico y reservamos para métricos error
forecast.mod1 = predict(modelo2,10)

<a id="15"></a> <br>
### H. Training model, con auto arima:

In [67]:
#Podemos aplicar el otro modelo (segundo mejor), a la sección de prueba para tener dos modelos a comparar 
#respecto al error de pronóstico:
mod2 = auto.arima(entrena)

<a id="16"></a> <br>
#### I. Model summary

In [68]:
#Analizamos modelo y residuales
summary(mod2)
tsdiag(mod2)

Todo se ve ok.
Vemos que partiendo la serie, el mejor modelo es un MA(1).

In [78]:
##Generamos igualmente su pronóstico para la parte de prueba:
forecast.modelo1_auto = predict(modelo1_auto,10)

<a id="18"></a> <br>
#### J. Evaluacion, con respecto a la muestra separada de valida.


In [79]:
rmse(valida, forecast.mod1$pred)
rmse(valida, forecast.modelo1_auto$pred)


Vemos que el segundo modelo presenta también los menores valores de error de pronóstico.

<a id="19"></a> <br>
#### K. Comparación con backtesting:

EN kaggle, requerimos "pegar" en una celda, la función de backtesting. Si estamos en Rstudio local, la llamamos mediante source("backtesting.R")

In [39]:
#backtest.R <- source(../input/backtestr/backtest.R)
"backtest" <- function(m1,rt,orig,h,xre=NULL,fixed=NULL,inc.mean=TRUE){
  # m1: is a time-series model object
  # orig: is the starting forecast origin
  # rt: the time series
  # xre: the independent variables
  # h: forecast horizon
  # fixed: parameter constriant
  # inc.mean: flag for constant term of the model.
  #
  regor=c(m1$arma[1],m1$arma[6],m1$arma[2])
  seaor=list(order=c(m1$arma[3],m1$arma[7],m1$arma[4]),period=m1$arma[5])
  T=length(rt)
  if(!is.null(xre) && !is.matrix(xre))xre=as.matrix(xre)
  ncx=ncol(xre)
  if(orig > T)orig=T
  if(h < 1) h=1
  rmse=rep(0,h)
  mabso=rep(0,h)
  nori=T-orig
  err=matrix(0,nori,h)
  jlast=T-1
  for (n in orig:jlast){
    jcnt=n-orig+1
    x=rt[1:n]
    if (!is.null(xre)){
      pretor=xre[1:n,]
      mm=arima(x,order=regor,seasonal=seaor,xreg=pretor,fixed=fixed,include.mean=inc.mean)
      nx=xre[(n+1):(n+h),]
      if(h==1)nx=matrix(nx,1,ncx)
      fore=predict(mm,h,newxreg=nx)
    }
    else {
      mm=arima(x,order=regor,seasonal=seaor,xreg=NULL,fixed=fixed,include.mean=inc.mean)
      fore=predict(mm,h,newxreg=NULL)
    }
    kk=min(T,(n+h))
    # nof is the effective number of forecats at the forecast origin n.
    nof=kk-n
    pred=fore$pred[1:nof]
    obsd=rt[(n+1):kk]
    err[jcnt,1:nof]=obsd-pred
  }
  #
  for (i in 1:h){
    iend=nori-i+1
    tmp=err[1:iend,i]
    mabso[i]=sum(abs(tmp))/iend
    rmse[i]=sqrt(sum(tmp^2)/iend)
  }
  print("RMSE of out-of-sample forecasts")
  print(rmse)
  print("Mean absolute error of out-of-sample forecasts")
  print(mabso)
  backtest <- list(origin=orig,error=err,rmse=rmse,mabso=mabso)
}

In [71]:
#Podemos también, aplicar la función de backtesting de Tsay para derivar el error.
h=10
bt.mode1=backtest(modelo2,r1,(l-h),1) #corresponde al modelo de mejor ajuste por AIC entre las opciones revisadas
bt.mod2=backtest(modelo1_auto,r1,(l-h),1)

A partir de estos resultados, vemos que el segundo modelo genera más valores extremos, de ahi que sea más penalizado mediante el RMSE en comparación con el primer modelo. Si lo comparamos con el error MAE, el segundo modelo es el más competitivo.
En resumen, el segundo modelo presenta mejores métricos de desempeño intramuestral, así como el menor desempeño mediante el error de MAE.

<a id="20"></a> <br>

### Pronóstico hacia adelante mejor modelo

In [81]:
# Generamos pronóstico del modelo 2: para ello, retomamos la serie completa y ajustamos modelo:
mod_fwd=Arima(r1, order=c(0,0,1), method="ML")
fwd_forecast = forecast(mod_fwd, h=10)
mod_fwd %>% autoplot() #para ver las raíces de los modelos. Se grafican las raíces de la ecuación caracterírstica, que sale de la transformación del polinómico característico.
fwd_forecast %>% autoplot(, include=100, main= "Pronóstico hacia adelante SPX")

Como conclusiones más relevantes; están los diferentes métodos para derivar modelos ARMA, donde el eacf() puede ser una herramienta muy valiosa para la lectura de los niveles p y q. Igualmente, contamos con la función auto,arima().
Finalmente, recalcar que la derivación del mejor modelo para la generación de pronósticos hacia adelante, usualmente se lleva a cabo partiendo la serie en un porcentaje alto de entrenamiento, se adapta el modelo y es el que se emplea para el pronóstico en la prueba. De ello, se selecciona el mejor que se aplicará a la serie total. En este tutorial vimos que el derivar modelos a la serie completa usualmente se realiza cuando no nos interesa una tarea de pronóstico, sino más de interpretación o de análisis de los features o rezagos más relevantes en la respuesta.